In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn, optim

:math:`\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}`


NumPy Arrays: NumPy arrays can be passed directly to TensorFlow/Keras functions. They will be automatically converted to TensorFlow tensors.
Automatic Differentiation: TensorFlow automatically tracks operations performed on tensors and provides automatic differentiation for computing gradients.


PyTorch: Eager execution by default. Operations are computed immediately, and the computational graph is dynamic.
Keras with TensorFlow 2.x: Eager execution is the default mode. However, it can also operate in graph mode by using tf.function to create a static computational graph if needed.



# Pytorch 

In [3]:
float_tensor = torch.zeros(3, 3, 3, dtype=torch.float32) + 1
print(float_tensor)

tensor([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]])


In [5]:
def channel_softmax(x):
    e = torch.exp(x - torch.max(x, axis=-1, keepdims=True)[0])
    s = torch.sum(e, axis=-1, keepdims=True)
    return e / s

x=torch.Tensor([[[3,3],[4,5]], [[4,4],[5,7]], [[5,5], [6,7]]])
print(x[:,1])
print(channel_softmax(float_tensor))
print(channel_softmax(torch.Tensor([4,5,6])))

tensor([[4., 5.],
        [5., 7.],
        [6., 7.]])
tensor([[[0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333]],

        [[0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333]],

        [[0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333]]])
tensor([0.0900, 0.2447, 0.6652])


In [64]:
channel_softmax = nn.Softmax(dim=-1)
output = channel_softmax(float_tensor)
print(output)

tensor([[[0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333]],

        [[0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333]],

        [[0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333],
         [0.3333, 0.3333, 0.3333]]])


In [6]:
def scale(x, v):
    return x / v



# Locally connected layer + Generator

In [7]:
from torch.nn.modules.utils import _pair

""" I modified locally connected2d so that bias is a float variable: if it is set to zero no bias is introduced, if it is set to 
any other number, that will be the bias
I also made another change: provide the input size together with all the other parameters of the convolution. It will authomatically 
compute the output dimension and prepare all the weights.
For now I assume that kernel size, padding, and stride are the same in both h and w dimensions 
In order to stack many of those layers I also return the output shape, which will be the input shape of the next layer
"""

""" #filters = #out_channels """

class LocallyConnected2d(nn.Module):
    def __init__(self, in_channels, out_channels, input_size, kernel_size, bias, stride=1, padding=0, dilation=1):
        super(LocallyConnected2d, self).__init__()
        self.kernel_size = _pair(kernel_size)
        self.stride = _pair(stride)
        output_size_0=int(1+ (input_size[0] + 2*padding - dilation*(kernel_size-1)-1)/self.stride[0])
        output_size_1=int(1+ (input_size[1] + 2*padding - dilation*(kernel_size-1)-1)/self.stride[1])
        self.weight = nn.Parameter(
            torch.randn(1, out_channels, in_channels, output_size_0, output_size_1, kernel_size**2)
        )
        if bias:
            self.bias = nn.Parameter(
                torch.full((1, out_channels, output_size_0, output_size_1), bias, requires_grad=True)
            )
        else:
            self.register_parameter('bias', None)
        
    def forward(self, x):
        _, c, h, w = x.size()
        kh, kw = self.kernel_size
        dh, dw = self.stride
        x = x.unfold(2, kh, dh).unfold(3, kw, dw)
        x = x.contiguous().view(*x.size()[:-2], -1)
        # Sum in in_channel and kernel_size dims       
        out = (x.unsqueeze(1) * self.weight).sum([2, -1])

        if self.bias is not None:
            out += self.bias 
        return out
    
        
   
batch_size = 5
in_channels = 3
h, w = 96,3
x = torch.randn(batch_size, in_channels, h, w)
print("x", x.shape)

# Create layer and test if backpropagation works

out_channels = 2
kernel_size = 2
stride = (3,1)


conv = LocallyConnected2d(
    in_channels, out_channels, [h,w], kernel_size, bias=6.0, stride=stride)
print("bias", conv.bias.shape)
out = conv(x)
print("check here", "out", out[0].shape)

class Generator(nn.Module):
    def __init__(self, latent_dim, nb_rows, nb_cols):
        super(Generator, self).__init__()
        self.fc = nn.Linear(latent_dim, (nb_rows + 2) * (nb_cols + 2) * 36)
        self.reshape_size = (nb_rows + 2, nb_cols + 2, 36)

        self.conv1 = nn.Conv2d(36, 16, kernel_size=2,padding='same')
        self.lrelu = nn.LeakyReLU()
        self.batch_norm = nn.BatchNorm2d(16)

        # Using custom LocallyConnected2d layers
        self.local_conv1 = LocallyConnected2d(16, 6, [nb_rows+2,nb_cols+2], kernel_size=2, bias=0)
        self.local_conv2 = LocallyConnected2d(6, 1, [nb_rows+1, nb_cols+1], kernel_size=2, bias=0)

    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, *self.reshape_size)  # (batch_size, nb_rows+2, nb_cols+2, 36)
        
        x = x.permute(0, 3, 1, 2)  # (batch_size, 36, nb_rows+2, nb_cols+2)
        x = self.conv1(x)
        x = self.lrelu(x)
        x = self.batch_norm(x)
        x = self.local_conv1(x)
        
        x = self.lrelu(x)
    
        x = self.local_conv2(x)
        return x


latent_dim = 100
nb_rows = 96
nb_cols = 96
model = Generator(latent_dim, nb_rows, nb_cols)

# Creare un input fittizio
dimensione_batch = 16
rumore = torch.randn(dimensione_batch, latent_dim)  # Batch di 16 vettori con dimensione latent_dim


# Generare immagini fake
immagini_fake = model(rumore)
print(immagini_fake.shape)


x torch.Size([5, 3, 96, 3])
bias torch.Size([1, 2, 32, 2])
check here out torch.Size([2, 32, 2])
torch.Size([16, 1, 96, 96])


/Users/martinachirico/miniconda3/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1004.)
  return F.conv2d(input, weight, bias, self.stride,


# Inpainting attention

In [ ]:
""" in InpaintingAttention ho tolto la funzione bias initializer perché mi sembra di averla implementata per bene  dentro il locally connected,
ho aggiunto come argomento della classe InpaintingAttention la dimensione di input"""


In [8]:
"""NOTA IMPORTANTE: per keras l'ultima dimensione è il canale, per pytorch il canale è la seconda dimensione"""
class InpaintingAttention(nn.Module):
    def __init__(self, constant=-10, input_size=[14,26]):
        super(InpaintingAttention, self).__init__()

        # Define zero padding layer
        self.pad = nn.ZeroPad2d((1, 1, 1, 1))
        

        # Define locally connected layer
        self.lcn = LocallyConnected2d(in_channels=2, out_channels=2, input_size=input_size, kernel_size=3, bias=constant, stride=1)

    def forward(self, primary, carryover):
        # Concatenate primary and carryover along the last dimension
        x = torch.cat((primary, carryover), dim=1)  #concatenate along channel dimension
        
        # Apply zero padding
        h = self.pad(x)
        
        # Apply locally connected layer
        h = self.lcn(h)

        # Compute weights using channel softmax
        weights = channel_softmax(h)
        
        # Compute the weighted sum
        weighted_sum = torch.sum(x * weights, dim=1, keepdim=True)

        return weighted_sum
           

model1=Generator(latent_dim, 3,96)
model2=Generator(latent_dim, 12,12)
model3=Generator(latent_dim, 12,6)
img_layer0 = model1(rumore)
img_layer1 = model2(rumore)
img_layer2 = model3(rumore)
no_attn=0

inpainting_attention1=InpaintingAttention(constant=-10.0, input_size=[14,14])
inpainting_attention2=InpaintingAttention(constant=-10.0, input_size=[14,8])

if not no_attn:

    # resizes from (3, 96) => (12, 12)
    zero2one = nn.AvgPool2d(kernel_size=(1, 8))(
            nn.Upsample(scale_factor=(4, 1), mode='nearest')(img_layer0)
        )
    img_layer1 = inpainting_attention1(img_layer1, zero2one)
    print("0->1",zero2one.shape, img_layer1.shape)

    # resizes from (12, 12) => (12, 6)
    one2two = nn.AvgPool2d(kernel_size=(1, 2))(img_layer1)

    img_layer2 = inpainting_attention2(img_layer2, one2two)
    print("1-2",one2two.shape,img_layer2.shape)
     




0->1 torch.Size([16, 1, 12, 12]) torch.Size([16, 1, 12, 12])
1-2 torch.Size([16, 1, 12, 6]) torch.Size([16, 1, 12, 6])


# Minibatch Discriminator

In [9]:
def minibatch_output_shape(input_shape):
    """ Computes output shape for a minibatch discrimination layer"""
    shape = list(input_shape)
    assert len(shape) == 3  # only valid for 3D tensors
    return tuple(shape[:2])
print(minibatch_output_shape((16,10,10)))

(16, 10)


In [10]:
def minibatch_discriminator(x):
    # Expand dimensions and compute differences
    diffs = x.unsqueeze(3) - x.permute(1, 2, 0).unsqueeze(0)
    #print(diffs.shape)

    # Compute the L1 norm
    l1_norm = torch.sum(torch.abs(diffs), dim=2)
    #print(l1_norm.shape)

    # Compute the exponent of the negative L1 norm and sum across the batch
    return torch.sum(torch.exp(-l1_norm), dim=2)


# Example usage
x = torch.randn(16,10,10)  # Example input tensor with shape [batch_size, channel, height, width]
output = minibatch_discriminator(x)
print(output.shape)

torch.Size([16, 10])


In [204]:
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
import numpy as np

def minibatch_discriminator(x):
    """ Computes minibatch discrimination features from input tensor x"""
    diffs = K.expand_dims(x, 3) - K.expand_dims(K.permute_dimensions(x, [1, 2, 0]), 0)
    l1_norm = K.sum(K.abs(diffs), axis=2)
    return K.sum(K.exp(-l1_norm), axis=2)

# Wrapper function to use with Keras Lambda layer
def minibatch_discriminator_wrapper(x):
    return minibatch_discriminator(x)

# Define input tensor
x = np.random.rand(16,10,10)  # Adjust input shape as needed
diffs = K.expand_dims(x, 3) - K.expand_dims(K.permute_dimensions(x, [1, 2, 0]), 0)
print("diffs", diffs.shape)
l1_norm = K.sum(K.abs(diffs), axis=2)
print("l1", l1_norm.shape)
print(K.sum(K.exp(-l1_norm), axis=2).shape)



diffs (16, 10, 10, 16)
l1 (16, 10, 16)
(16, 10)


' \n# Apply minibatch_discriminator using a Lambda layer\noutput_tensor = Lambda(minibatch_discriminator_wrapper)(input_tensor)\n\n# Create model\nmodel = Model(inputs=input_tensor, outputs=output_tensor)\n\n# Summary of the model\nmodel.summary()\n\n# Create some sample data\nbatch_size = 8\nsample_data = np.random.rand(batch_size, 32, 32, 3)  # Example batch of 8 images\n\n# Predict using the model\npredictions = model.predict(sample_data)\n\n# Print the shape of the output\nprint("Output shape:", predictions.shape)\nprint(predictions) '

# Sparsity level

In [13]:
#questa funzione è indipendente dalla convenzione sulla posizione del canale
def sparsity_output_shape(input_shape):
    shape = list(input_shape)
    return (shape[0], 1)

print(sparsity_output_shape([16,10,10, 1]))

(14, 1)


In [15]:
def sparsity_level(x):
    # Get the shape of the input tensor
    batch_size, channels, height, width = x.shape
    
    # Calculate the total number of elements per sample (ignoring batch dimension)
    total = float(channels * height * width)
    
    # Count the number of non-zero elements along the (channels, height, width) dimensions
    non_zero_counts = (x > 0.0).float().sum(dim=[1, 2, 3])
    
    # Reshape to a column vector [batch_size, 1]
    non_zero_counts = non_zero_counts.view(batch_size, 1)
    
    # Calculate the sparsity level
    sparsity = non_zero_counts / total
    
    return sparsity

# Example usage:
x = torch.randn(16,1,10,10)  # Example tensor with shape [batch_size, channels, height, width]
sparsity = sparsity_level(x)
print(sparsity.shape)  # Output shape should be [32, 1]


torch.Size([16, 1])


# Dense 3D layer 

In [17]:
""" Note that in PyTorch, constraints and regularizers are typically handled separately, 
and the above example assumes simple weight initialization without custom constraints or regularizers. I
changed the settings so that input_shape is a compulsory argument. I was not able to manage **kwargs
 """

class Dense3D(nn.Module):
    """
    A 3D, trainable, dense tensor product layer.
    """

    def __init__(self, first_dim, last_dim, input_shape, activation=None, use_bias=True):
        super(Dense3D, self).__init__()
        self.first_dim = first_dim
        self.last_dim = last_dim
        self.activation = activation if activation else lambda x: x
        self.use_bias = use_bias
        self.input_shape = input_shape
        self.input_dim = self.input_shape[-1]
        # Apply constraints and regularizers (if any) after initialization
        
        # Initialize kernel (weights)
        assert len(input_shape) >= 2
        self.kernel = nn.Parameter(
            torch.randn(first_dim, self.input_dim, last_dim)
        )
        nn.init.xavier_uniform_(self.kernel)
   
        # Initialize bias if use_bias is True
        if use_bias:
            self.bias = nn.Parameter(torch.zeros(first_dim, last_dim))
        else:
            self.bias = None
        
        nn.init.zeros_(self.bias)
        

    def forward(self, inputs):
        # Compute the output
        print("Dense3D: input:", inputs.shape, "kernel:",self.kernel.shape)
        out = torch.tensordot(inputs, self.kernel, dims=([1],[1]))
        if self.use_bias:
            out += self.bias
        # Apply activation function
        if self.activation:
            out = self.activation(out)

        return out

    def compute_output_shape(self, input_shape):
        assert len(input_shape) == 2
        return (input_shape[0], self.first_dim, self.last_dim)


# Example usage
input_tensor = torch.randn(10, 5)
model = Dense3D(first_dim=3, last_dim=4, activation=F.relu, input_shape=(10,5))
output = model(input_tensor)
print(output.shape)
print(model.compute_output_shape( (10,5) ) )



Dense3D: input: torch.Size([10, 5]) kernel: torch.Size([3, 5, 4])
torch.Size([10, 3, 4])
(10, 3, 4)


# Discriminator

In [19]:
class Discriminator(nn.Module):
    def __init__(self, mbd=False, sparsity=False, sparsity_mbd=False):
        super(Discriminator, self).__init__()
        self.mbd = mbd
        self.sparsity = sparsity
        self.sparsity_mbd = sparsity_mbd

        self.conv1 = nn.Conv2d(1, 64, kernel_size=2, padding='same')
        self.lrelu = nn.LeakyReLU()

        self.zero_pad = nn.ZeroPad2d(1)
        self.local_conv1 = LocallyConnected2d(64, 16, input_size=(12, 12), kernel_size=3, stride=(1,2), bias=0)
        self.batch_norm1 = nn.BatchNorm2d(16)

        self.local_conv2 = LocallyConnected2d(16, 8, input_size=(12, 7), kernel_size=2, bias=0)
        self.batch_norm2 = nn.BatchNorm2d(8)

        self.local_conv3 = LocallyConnected2d(8, 8, input_size=(13, 8), kernel_size=2, stride=(1,2), bias=0)
        self.batch_norm3 = nn.BatchNorm2d(8)

        self.flatten = nn.Flatten()

        if self.mbd or self.sparsity or self.sparsity_mbd:
            self.minibatch_featurizer = minibatch_discriminator
            nb_features = 10
            vspace_dim = 10
            self.dense3d_1 = Dense3D(first_dim=nb_features, last_dim=vspace_dim, input_shape=(16,384) )
            self.dense3d_2 = Dense3D(first_dim=nb_features, last_dim=vspace_dim, input_shape=(16,1))
            self.activation_tanh = nn.Tanh()
            self.sparsity_detector = sparsity_level 

    def forward(self, image):
        #x = image.permute(0, 3, 1, 2)
        x=image
        x = self.conv1(x)
        x = self.lrelu(x)

        x = self.zero_pad(x) 

        x = self.local_conv1(x)
        x = self.lrelu(x)
        x = self.batch_norm1(x)

        x = self.zero_pad(x)
        print("after loc2", x.shape)
        
        x = self.local_conv2(x)
        x = self.lrelu(x)
        x = self.batch_norm2(x)

        x = self.zero_pad(x)
    
        x = self.local_conv3(x)
        x = self.lrelu(x)
        x = self.batch_norm3(x)

        x = self.flatten(x)
        print("before self.mbd", x.shape)
        if self.mbd or self.sparsity or self.sparsity_mbd:
            features = [x]
            if self.mbd:
                K_x = self.dense3d_1(x)
                print("K_x:", K_x.shape)
                print("after minibatch:", self.minibatch_featurizer(K_x).shape)
                features.append(self.activation_tanh(self.minibatch_featurizer(K_x)))
                #print("features:", features.shape)

            if self.sparsity or self.sparsity_mbd:
                empirical_sparsity = self.sparsity_detector(image)
                if self.sparsity:
                    features.append(empirical_sparsity)
                if self.sparsity_mbd:
                    K_sparsity = self.dense3d_2(empirical_sparsity)
                    features.append(self.activation_tanh(self.minibatch_featurizer(K_sparsity)))

            return torch.cat(features, dim=1)
        else:
            return x
    
discriminator = Discriminator(mbd=False, sparsity=False, sparsity_mbd=True)
batch_size = 16
input_channels = 1
height = 10
width = 10
input_tensor = torch.randn(batch_size,input_channels, height, width)
print("input", input_tensor.shape)

# Passare l'input attraverso il modello
output_tensor = discriminator(input_tensor)
print("output:", output_tensor.shape)

input torch.Size([16, 1, 10, 10])
after loc2 torch.Size([16, 16, 12, 7])
before self.mbd torch.Size([16, 384])
Dense3D: input: torch.Size([16, 1]) kernel: torch.Size([10, 1, 10])
output: torch.Size([16, 394])


In [ ]:
# prima di spostare il canale prima di darlo alla rete
input torch.Size([16, 10, 10, 1])
after loc2 torch.Size([16, 16, 12, 7])
before self.mbd torch.Size([16, 384])
Dense3D: input: torch.Size([16, 384]) kernel: torch.Size([10, 384, 10])
K_x: torch.Size([16, 10, 10])
after minibatch: torch.Size([16, 10])
output: torch.Size([16, 394])

# Energy error and other functions 

In [96]:
def energy_error(requested_energy, received_energy):
    # Compute the difference
    difference = (received_energy - requested_energy) / 10000

    # Determine if the energy is over the requested amount
    over_energized = (difference > 0).float()
    print(over_energized)

    # Compute the penalties for too high and too low energy
    too_high = 100 * torch.abs(difference)
    too_low = 10 * torch.abs(difference)

    # Compute the final energy error
    return over_energized * too_high + (1 - over_energized) * too_low

# Example usage:
requested_energy = torch.tensor([100.0, 150.0, 200.0])
received_energy = torch.tensor([110.0, 140.0, 210.0])
error = energy_error(requested_energy, received_energy)
print(error)

tensor([1., 0., 1.])
tensor([0.1000, 0.0100, 0.1000])


In [ ]:
def single_layer_energy(x):
    shape = x.shape
    return torch.sum(x, dim=tuple(range(1, len(shape)))).view(-1, 1)


# Trash

In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the custom layer
class ChannelSoftmax(nn.Module):
    def forward(self, x):
        e = torch.exp(x - torch.max(x, dim=-1, keepdim=True)[0])
        s = torch.sum(e, dim=-1, keepdim=True)
        return e / s

# Build a simple model
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc = nn.Linear(10, 32)  # Dense layer with 10 input features and 32 output features
        self.softmax = ChannelSoftmax()  # Custom softmax layer

    def forward(self, x):
        x = torch.relu(self.fc(x))  # ReLU activation after the dense layer
        x = self.softmax(x)  # Apply custom softmax
        return x

# Create an instance of the model
model = Model()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification problems
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with a learning rate of 0.001

# Generate some dummy data
x_train = torch.randn(100, 10)  # Random input data with shape (100, 10)
y_train = torch.randint(0, 32, (100,))  # Random labels with values between 0 and 31 (inclusive)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(x_train)
    # Compute loss
    loss = criterion(outputs, y_train)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # Print progress
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 3.4654
Epoch [2/10], Loss: 3.4653
Epoch [3/10], Loss: 3.4652
Epoch [4/10], Loss: 3.4650
Epoch [5/10], Loss: 3.4649
Epoch [6/10], Loss: 3.4648
Epoch [7/10], Loss: 3.4646
Epoch [8/10], Loss: 3.4645
Epoch [9/10], Loss: 3.4644
Epoch [10/10], Loss: 3.4643


# Keras

## How to define a model in keras. Three ways:

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the model using the functional API
inputs = tf.keras.Input(shape=(10,))
x = layers.Dense(5, activation='relu')(inputs)
x = layers.Dense(1)(x)
outputs = layers.Activation('sigmoid')(x)

# Create the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation

# Define the model using the sequential API
model = Sequential([
    Dense(5, input_shape=(10,), activation='relu'),
    Dense(1),
    Activation('sigmoid')
])

import tensorflow as tf
from tensorflow.keras import layers, Model

# Define the model as a class
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = layers.Dense(5, activation='relu')
        self.dense2 = layers.Dense(1)
        self.activation = layers.Activation('sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.activation(x)

# Instantiate the model
model = MyModel()

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras import backend as K

# Define the custom channel_softmax function
def channel_softmax(x):
    e = K.exp(x - K.max(x, axis=-1, keepdims=True))
    s = K.sum(e, axis=-1, keepdims=True)
    return e / s

# Create an input tensor
input_tensor = Input(shape=(10,))

# Define a simple model with a dense layer followed by the custom softmax layer
x = Dense(32, activation='relu')(input_tensor)
x = Lambda(channel_softmax)(x)

# Create the model
model = Model(inputs=input_tensor, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Print the model summary
model.summary()

# Generate some dummy data
import numpy as np
x_train = np.random.rand(100, 10)
y_train = np.random.randint(2, size=(100, 32))

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=16)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 dense (Dense)               (None, 32)                352       
                                                                 
 lambda (Lambda)             (None, 32)                0         
                                                                 
Total params: 352 (1.38 KB)
Trainable params: 352 (1.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
7/7 [==============================] - 0s 1ms/step - loss: 54.7312
Epoch 2/10
7/7 [==============================] - 0s 733us/step - loss: 54.6636
Epoch 3/10
7/7 [==============================] - 0s 918us/step - loss: 54.6089
Epoch 4/10
7/7 [==============================] - 0s 918u

In [103]:
import numpy as np
from keras.layers import concatenate, ZeroPadding2D, Lambda, multiply, LocallyConnected2D
from keras.initializers import Constant
from keras.models import Model
from keras import backend as K

def inpainting_attention(primary, carryover, constant=-10):

    def _initialize_bias(const=-5):
        def _(shape, dtype=None):
            assert len(shape) == 3, 'must be a 3D shape'
            x = np.zeros(shape)
            x[:, :, -1] = const
            return x
        return _

    x = concatenate([primary, carryover], axis=-1)
    h = ZeroPadding2D((1, 1))(x)
    lcn = LocallyConnected2D(
        filters=2,
        kernel_size=(3, 3),
        bias_initializer=_initialize_bias(constant)
    )

    h = lcn(h)
    weights = Lambda(channel_softmax)(h)

    channel_sum = Lambda(K.sum, arguments={'axis': -1, 'keepdims': True})

    return channel_sum(multiply([x, weights]))


# Example usage
# Create sample primary and carryover tensors
primary = np.random.randn(1, 32, 32, 3)  # Sample primary tensor with shape (1, 32, 32, 3)
carryover = np.random.randn(1, 32, 32, 3)  # Sample carryover tensor with shape (1, 32, 32, 3)

# Set constant value for bias initialization
constant_value = -10

# Call the inpainting_attention function
output = inpainting_attention(primary, carryover, constant=constant_value)

# Print the shape of the output tensor
print("Output shape:", output.shape)

ValueError: Inputs have incompatible shapes. Received shapes (32, 32, 6) and (32, 32, 2)

In [107]:

def _(shape, dtype=None):
    assert len(shape) == 3, 'must be a 3D shape'
    x = np.zeros(shape)
    x[:, :, -1] = 1
    return x


print(_([3,3,3]))

[[[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]

 [[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]]]


In [ ]:
from torch.nn.modules.utils import _pair

""" I modified locally connected2d so that bias is a float variable: if it is set to zero no bias is introduced, if it is set to 
any other number, that will be the bias
I also made another change: provide the input size together with all the other parameters of the convolution. It will authomatically 
compute the output dimension and prepare all the weights.
For now I assume that kernel size, padding, and stride are the same in both h and w dimensions 
In order to stack many of those layers I also return the output shape, which will be the input shape of the next layer
"""

""" #filters = #out_channels """

class LocallyConnected2d(nn.Module):
    def __init__(self, in_channels, out_channels, input_size, kernel_size, bias, stride=1, padding=0, dilation=1):
        super(LocallyConnected2d, self).__init__()
        output_size_0=int(1+ (input_size[0] + 2*padding - dilation*(kernel_size-1)-1)/stride)
        output_size_1=int(1+ (input_size[1] + 2*padding - dilation*(kernel_size-1)-1)/stride)
        self.weight = nn.Parameter(
            torch.randn(1, out_channels, in_channels, output_size_0, output_size_1, kernel_size**2)
        )
        if bias:
            self.bias = nn.Parameter(
                torch.full((1, out_channels, output_size_0, output_size_1), bias, requires_grad=True)
            )
        else:
            self.register_parameter('bias', None)
        self.kernel_size = _pair(kernel_size)
        self.stride = _pair(stride)

    def forward(self, x):
        _, c, h, w = x.size()
        kh, kw = self.kernel_size
        dh, dw = self.stride
        x = x.unfold(2, kh, dh).unfold(3, kw, dw)
        x = x.contiguous().view(*x.size()[:-2], -1)
        # Sum in in_channel and kernel_size dims       
        out = (x.unsqueeze(1) * self.weight).sum([2, -1])

        if self.bias is not None:
            out += self.bias 
        return out, out.shape
    
        
   
batch_size = 5
in_channels = 3
h, w = 96,3
x = torch.randn(batch_size, in_channels, h, w)
print("x", x.shape)

# Create layer and test if backpropagation works

out_channels = 2
kernel_size = 3
stride = 1


conv = LocallyConnected2d(
    in_channels, out_channels, [h,w], kernel_size, bias=6.0, stride=stride)
print("bias", conv.bias.shape)
out = conv(x)
print("out", out[0].shape)
